In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles

In [2]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/";

In [3]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC31.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity31.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance31.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"), header=1, delim=",") |> DataFrame 
SDD = CSV.File(string(path,"SDD31.csv"),header=1,delim=",") |> DataFrame;

In [4]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)                          # 2021 global GD ($/yr)
es_ratio = global_sink/global_emi

emission_c = SDD[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = SDD[!, "sink ton/yr"]           # national CO2 sink (ton/yr)
Dsoc = SDD[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"];

In [5]:
cell_demand = 0.001*164.98*(1.369*1e6)*2;           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 164.98 kg/pack (35kwh/pack), 80~100 kWh per EV)

In [67]:
emission_c

31-element Vector{Float64}:
 4.91e8
 7.55e8
 2.71e9
 1.13e9
 1.54e8
 1.26e10
 1.64e8
 4.95e7
 4.77e7
 2.39e8
 8.31e8
 3.42e9
 2.03e9
 ⋮
 5.8e7
 1.47e8
 2.7e8
 8.33e7
 3.2e9
 5.24e8
 4.46e8
 7.23e8
 3.68e8
 2.95e8
 1.89e8
 6.84e9

In [72]:
test = ones(10,1)
test[2]

1.0

----

In [79]:
function OvershootNum(tp="PRODUCER")
    model = Model(Gurobi.Optimizer)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, delta[1:ncty], Bin)
    
    M = 1e10
    slack = 1e-10
    
    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)


    transD = Matrix{AffExpr}(undef, ncty, nproc)
    if tp == "PRODUCER"
        for k in 1:nproc
            for j in 1:ncty
                emi = 0
                for i in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
                end
                transD[j,k] = emi
            end
        end
    elseif tp == "CUSTOMER"
        for k in 1:nproc
            for i in 1:ncty
                emi = 0
                for j in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans #CUSTOMER
                end
                transD[i,k] = emi
            end
        end
    end
    
    Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
    SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
    cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty]

    EO = (proD - proS + transD) * ones(nproc,1)
    
    for i in 1:ncty
        @constraint(model, EO[i] >= slack + (delta[i] - 1) * M)
        @constraint(model, EO[i] <= delta[i] * M - slack)
    end
    
    @objective(model, Min, sum(delta))
    JuMP.optimize!(model)
    
    optx = JuMP.value.(x)
    opty = JuMP.value.(y)
    obj = JuMP.objective_value(model)
    res = Dict(["optx"=>optx, "optobj"=>obj, "opty"=>opty])
    
    return res
end

OvershootNum (generic function with 2 methods)

In [80]:
OvershootNum("PRODUCER")

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1502 rows, 14911 columns and 74369 nonzeros
Model fingerprint: 0x5ab99c85
Variable types: 14880 continuous, 31 integer (31 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-10, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1184 rows and 13368 columns
Presolve time: 0.03s
Presolved: 318 rows, 1543 columns, 4559 nonzeros
Variable types: 1517 continuous, 26 integer (26 binary)

Root relaxation: objective 2.017784e+00, 54 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap |

Dict{String, Any} with 3 entries:
  "optobj" => 3.0
  "opty"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…
  "optx"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 900…

In [81]:
OvershootNum("CUSTOMER")

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1502 rows, 14911 columns and 74369 nonzeros
Model fingerprint: 0xebe21b55
Variable types: 14880 continuous, 31 integer (31 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-10, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1180 rows and 13352 columns
Presolve time: 0.01s
Presolved: 322 rows, 1559 columns, 4831 nonzeros
Variable types: 1530 continuous, 29 integer (29 binary)

Root relaxation: objective 1.537068e+00, 234 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 

Dict{String, Any} with 3 entries:
  "optobj" => 2.0
  "opty"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…
  "optx"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…

In [82]:
function OvershootNum_proc(tp="PRODUCER")
    model = Model(Gurobi.Optimizer)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, delta[1:ncty, 1:nproc], Bin)
    
    M = 1e10
    slack = 1e-10
    
    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)


    transD = Matrix{AffExpr}(undef, ncty, nproc)
    if tp == "PRODUCER"
        for k in 1:nproc
            for j in 1:ncty
                emi = 0
                for i in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
                end
                transD[j,k] = emi
            end
        end
    elseif tp == "CUSTOMER"
        for k in 1:nproc
            for i in 1:ncty
                emi = 0
                for j in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans #CUSTOMER
                end
                transD[i,k] = emi
            end
        end
    end
    
    Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
    SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
    cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty]

    EO = proD - proS + transD
    
    for i in 1:ncty
        for k in 1:nproc
            @constraint(model, EO[i,k] >= slack + (delta[i,k] - 1) * M)
            @constraint(model, EO[i,k] <= delta[i,k] * M - slack)
        end
    end
    
    @objective(model, Min, sum(delta))
    JuMP.optimize!(model)
    
    optx = JuMP.value.(x)
    opty = JuMP.value.(y)
    obj = JuMP.objective_value(model)
    res = Dict(["optx"=>optx, "optobj"=>obj, "opty"=>opty])
    
    return res
end

OvershootNum_proc (generic function with 2 methods)

In [84]:
OvershootNum_proc("PRODUCER")

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2370 rows, 15345 columns and 75237 nonzeros
Model fingerprint: 0xb38b9931
Variable types: 14880 continuous, 465 integer (465 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-10, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1899 rows and 13697 columns
Presolve time: 0.02s
Presolved: 471 rows, 1648 columns, 4655 nonzeros
Variable types: 1458 continuous, 190 integer (190 binary)

Root relaxation: objective 7.258539e+00, 560 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   

Dict{String, Any} with 3 entries:
  "optobj" => 17.0
  "opty"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…
  "optx"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…

In [85]:
OvershootNum_proc("CUSTOMER")

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2370 rows, 15345 columns and 75237 nonzeros
Model fingerprint: 0xb92de35b
Variable types: 14880 continuous, 465 integer (465 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-10, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1906 rows and 13679 columns
Presolve time: 0.03s
Presolved: 464 rows, 1666 columns, 4742 nonzeros
Variable types: 1512 continuous, 154 integer (154 binary)

Root relaxation: objective 4.734132e+00, 655 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   

Dict{String, Any} with 3 entries:
  "optobj" => 8.0
  "opty"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 49670.3; … ; 0.0 0.0 … 0.0 0.0;…
  "optx"   => [0.0 0.0 … 0.0 0.0; 49670.3 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0;…

In [111]:
function OvershootPct_proc(tp="PRODUCER")
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "NonConvex", 2)
    
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, delta[1:ncty, 1:nproc], Bin)
    @variable(model, z[1:ncty, 1:nproc], Bin)
    @variable(model, obj>=0);
    
    M = 1e10
    slack = 1e-10
    
    x_cth = [@constraint(model, sum(x[i,k]*z[i,k] for i in 1:ncty) == sum(x[i,5]*z[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k]*z[i,k] for i in 1:ncty) == sum(x[i,11]*z[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k]*z[i,k] for i in 1:ncty) == sum(x[i,14]*z[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k]*z[i,k] for i in 1:ncty) == sum(x[i,15]*z[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k]*z[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]*z[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*z[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*z[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*z[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*z[j,15]*scaler[k]) for k in battery for j in 1:ncty]

    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    proD = x .* z .* Matrix(regional_EF[:,2:end])
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* z .* pro_sink


    transD = Matrix{AffExpr}(undef, ncty, nproc)
    if tp == "PRODUCER"
        for k in 1:nproc
            for j in 1:ncty
                emi = 0
                for i in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
                end
                transD[j,k] = emi
            end
        end
    elseif tp == "CUSTOMER"
        for k in 1:nproc
            for i in 1:ncty
                emi = 0
                for j in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans #CUSTOMER
                end
                transD[i,k] = emi
            end
        end
    end
    
    Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
    SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
    cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty]

    EO = proD - proS + transD
    
    for i in 1:ncty
        for k in 1:nproc
            @constraint(model, EO[i,k] >= slack + (delta[i,k] - 1) * M)
            @constraint(model, EO[i,k] <= delta[i,k] * M - slack)
        end
    end
    
    
    @constraint(model, obj*(sum(z)) == sum(delta))
    @objective(model, Min, obj)
    
#     obj = @expression(model, sum(delta[i,k] for i in 1:ncty for k in 1:nproc)/sum(z[i,k] for i in 1:ncty for k in 1:nproc))
#     @objective(model, Min, obj)
# #     @NLobjective(model, Min, sum(delta[i,k] for i in 1:ncty for k in 1:nproc)/sum(z[i,k] for i in 1:ncty for k in 1:nproc))
    
    
    JuMP.optimize!(model)
    optx = JuMP.value.(x)
    opty = JuMP.value.(y)
    optz = JuMP.value.(z)
    obj = JuMP.objective_value(model)
    res = Dict(["optx"=>optx, "optobj"=>obj, "opty"=>opty, "optz"=>optz])
    
    return res
end



OvershootPct_proc (generic function with 2 methods)

In [105]:
OvershootPct_proc()

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 31 rows, 15811 columns and 961 nonzeros
Model fingerprint: 0xe7dbc982
Model has 2340 quadratic constraints
Variable types: 14881 continuous, 930 integer (930 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [4e-03, 9e+01]
  QLMatrix range   [7e-02, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+05, 5e+05]
  QRHS range       [1e-10, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve added 482 rows and 0 columns
Presolve removed 0 rows and 12880 columns
Presolve time: 0.03s
Presolved: 2566 rows, 5721 columns, 11584 nonzeros
Presolved model has 2147 SOS constraint(s)
Variable types: 3869 continuous, 1852 integer 

Dict{String, Any} with 4 entries:
  "optobj" => 0.0580645
  "optz"   => [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0…
  "opty"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…
  "optx"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 4.51715e5 0.0; … ; 0.0 0.0 … 0.0 0.…

---

In [43]:
model = Model(Gurobi.Optimizer)
@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, delta[1:ncty], Bin)

x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
for j in tmp
    @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
end

Academic license - for non-commercial use only - expires 2024-12-26


In [44]:
proD = (x .* Matrix(regional_EF[:,2:end])) 
pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink) # ncty*1 matrix


transD = Matrix{AffExpr}(undef, ncty, nproc)

for k in 1:nproc
    for j in 1:ncty
        emi = 0
        for i in 1:ncty
            emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
        end
        transD[j,k] = emi
    end
end

# for k in 1:nproc
#     for i in 1:ncty
#         emi = 0
#         for j in 1:ncty
#             emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans #CUSTOMER
#         end
#         transD[i,k] = emi
#     end
# end


EO = (proD - proS + transD) * ones(nproc,1);

In [45]:
M = 1e10
slack = 1e-10
for i in 1:ncty
    @constraint(model, EO[i] >= slack + (delta[i] - 1) * M)
    @constraint(model, EO[i] <= delta[i] * M - slack)
end

In [46]:
@objective(model, Min, sum(delta));

In [47]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1471 rows, 14911 columns and 59954 nonzeros
Model fingerprint: 0x4c8e3449
Variable types: 14880 continuous, 31 integer (31 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-10, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1153 rows and 13368 columns
Presolve time: 0.03s
Presolved: 318 rows, 1543 columns, 4559 nonzeros
Variable types: 1517 continuous, 26 integer (26 binary)

Root relaxation: objective 2.017784e+00, 54 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.01778    0    1          -    2.017

In [48]:
xval = JuMP.value.(x)
yval = JuMP.value.(y);

In [49]:
proD = (xval .* Matrix(regional_EF[:,2:end])) 
pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (xval .* pro_sink) # ncty*1 matrix


transD = zeros(ncty, nproc)

for k in 1:nproc
    for j in 1:ncty
        emi = 0
        for i in 1:ncty
            emi += yval[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
        end
        transD[j,k] = emi
    end
end

In [50]:
(proD - proS + transD) * ones(nproc,1)

31×1 Matrix{Float64}:
       0.0
       0.0
       0.0
    -594.2910576851282
  -78029.361656298
  163580.65077501818
 -371643.0698418264
  353289.8136889406
       0.0
       0.0
       0.0
       0.0
       0.0
       ⋮
       0.0
 -131886.98926468496
       0.0
       0.0
       0.0
       0.0
       0.0
       0.0
       0.0
       0.0
       0.0
       3.530211104950824e6

In [56]:
SDD
TEST = SDD[:, "Dsoc ton/yr"] - SDD[:, "emission"]

for d in TEST
    if d > 0 
        println(d)
    end
end